<a href="https://colab.research.google.com/github/ulbin67/htmlpractice1/blob/master/2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-on LLM 실습

---

* 기초적인 LLM 문법과 이론에서 나온 것을 직접 코딩해보기


## 1장: 대규모 언어 모델 소개

---

* 간단한 NLP, RNN, 어텐션 모델로의 발전 과정과 역사를 살펴본다.
    - BoW -> word2vec -> transformer 로 발전
    - 언어 사전(머신러닝), 딥러닝+RNN, 어텐션
* 어텐션 모델의 개념에 대해 살펴본다.
* 파운데이션 모델과 독점 모델(closed source model)의 차이점을 알아본다.

In [1]:
%pip install --upgrade transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [3]:
from transformers import pipeline

#파이프라인을 만든다.
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)
#두 샘플 = 모델이 다

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
#프롬프트 (사용자 입력/쿼리)
messages = [
    {"role":"user","content":"Create a funny joke about Chickens."}
    ]

#출력 생성
output = generator(messages)
print(output[0]["generated_text"])

 Why don't chickens use computers? Because they're afraid of the "bro-d" and the "crow-d"!


## 2장: 토큰과 임베딩

---

* 토크나이저 선택 방식을 알아본다.
    - 1. 모델 설계 시에 모델 작성자가 토큰화 방법을 선택한다.
    - 2. 토큰화 방법을 선택한 후에 어휘사전 크기와 특수 토큰 같은 토크나이저 설계 상의 여러가지 선택을 한다.
    - 3. 토크나이저를 특정 데이터셋에서 훈련하여 해당 데이터셋을 표현하는 최상의 어휘사전을 구축하도록 한다.
* 토큰화 기법의 차이점을 알아본다
    - 단어 토큰, 부분단어 토큰, 문자 토큰, 바이트 토큰
    - <- 단어의 접미사 같은 문제 해결이 어려움 / 문맥 길이를 많이 소모 ->

In [6]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happend.<|assistant|>"

#입력 프롬프트를 토큰으로 나눕니다
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

#텍스트를 생성합니다.
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=20
)

print(tokenizer.decode(generation_output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happend.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap


Dear


In [7]:
input_ids

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  2250,   355, 29889, 32001]], device='cuda:0')

In [8]:
for id in input_ids[0]:
    print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happ
end
.
<|assistant|>


In [10]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  2250,   355, 29889, 32001,  3323,   622, 29901,   317,  3742,
           406,  6225, 11763,   363,   278, 19906,   292,   341,   728,   481,
            13,    13,    13, 29928,   799]], device='cuda:0')

In [11]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


### 토크나이저들의 각 차이점

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

#토큰끼리 색을 달리하여 표현하는 함수
def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [13]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

#### 1. BERT 베이스 모델 (uncased)

In [14]:
show_tokens(text, "bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

줄바꿈이 사라짐, 대문자를 구별하지 않음

"##" 을 통해 부분 토큰을 확인할 수 있음

"[UNK]" 알 수 없음 존재

#### 2. BERT 베이스 모델 (cased)



In [15]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

uncased보다 더 많이 토큰이 나눠진걸 볼 수 있다. 접미나 접두사를 많이 나누었음

#### 3. GPT-2

In [16]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

이모지와 한자가 각 다른 토큰으로 구분됨(문자가 깨진 것이지 실제로는 구분중)

줄바꿈이 표현됨

tabs는 탭을 표현하는 토큰과 공백 토큰, " 토큰으로 나누어짐

#### 4. Flan-T5

In [17]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

줄바꿈, 공백이 없음 (모델이 다루기 어려워짐)

#### 5. GPT-4

In [18]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

4개 공백을 하나의 토큰으로 표현함.

파이썬 키워드인 elif를 하나의 토큰으로 처리함. (코드에도 초점을 맞추고 있음)

CAPITALIZATION의 분할 방법을 보면 더 적은 토큰을 이용함

#### 6. StarCoder2

In [19]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

수학과 코드를 더 잘 이해하기 위한 방법들이 보임

숫자를 하나씩 처리하거나 코드도 하나의 토큰으로 처리됨

#### 7. Galactica

In [20]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

과학 지식에 초점을 맞추어 학습됨 위 토크나이저랑 비슷함

#### 8. Phi-3

In [21]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

대화 차례와 화자의 역할을 나타내는 특수 토큰이 추가됨

---

### 토크나이저 속성

* 토큰화 방법
    - 가장 인기가 높은 토큰화 방법은 BPE(GPT가 사용)
    - 허깅 페이스 토크나이저 요약 페이지 참고

* 토크나이저 파라미터
    - 어휘사전 크기
    - 특수 토큰 : 특수한 목적을 가지고 LLM을 만드려면 모데인을 잘 모델링할 수 있는 토큰 추가
    - 대소문자

* 데이터의 도메인
    - 동일한 방법과 파라미터를 선택했더라도 훈련한 데이터셋에 따라 토크나이저의 동작이 달라짐